In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from functions import analyse_new_columns, feature_engineering, one_hot, prepare_data, cv, \
    test_model
%load_ext autoreload
%autoreload 2
import lightfm
from lightfm import LightFM
from lightfm.evaluation import auc_score


In [2]:
data = pd.read_csv("/Users/victorialokteva/Downloads/data.csv")
# выкинем колонки, которые не нужы для 2 го задания, а также impressions
data = data.drop(columns=['banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1'])
data = data.drop(columns=['impressions'])
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,1


In [3]:
# Появились новые колонки в таблице по сравнению с первым заданием. Проанализируем их
analyse_new_columns(data)

Количество записей в таблице:  15821472

Новые колонки
Количество уникальных значений для oaid_hash:  6510316
Nan в oaid_hash:  False
Встречаемость самого частого значения: 5243
Встречаемость самого редкого значения: 1


Построение модели

In [4]:
 # добавим фич и сделаем one-hot encoding
data = feature_engineering(data)
data = one_hot(data)

In [5]:
data.columns

Index(['date_time', 'zone_id', 'banner_id', 'oaid_hash', 'campaign_clicks',
       'country_id', 'clicks', 'time', 'date', 'is_weekend', 'month',
       'date_copy', 'os_id_1', 'os_id_2', 'os_id_3', 'os_id_4', 'os_id_5',
       'os_id_6', 'os_id_7', 'os_id_8', 'os_id_9', 'os_id_10',
       'time_of_day_1.0', 'time_of_day_2.0', 'time_of_day_3.0', 'zone_17',
       'zone_14', 'zone_12', 'zone_0', 'zone_19', 'zone_1', 'zone_27',
       'zone_30', 'zone_50', 'zone_2', 'country_0', 'country_5', 'country_6',
       'country_7', 'country_1', 'country_3', 'country_15', 'country_12',
       'country_9', 'country_4', 'country_10'],
      dtype='object')

In [6]:
# Разобъем на тест и тест: в качестве теста возьмем последний день
# Данные по юзеру и айтему проктнем отдельно от трэйна
user_item_train, user_item_test, user_feat_train, item_feat_train, user_feat_test, item_feat_test  = prepare_data(data)


In [13]:
# Подберем параметр, отвечающий за размерность
n_factors = cv(user_item_train, user_feat_train, item_feat_train)

AttributeError: module 'lightfm' has no attribute 'cross_validation'

In [ ]:
print(n_factors)

In [11]:
model = LightFM(no_components=10)

model.fit(interactions=user_item_train,
          user_features=user_feat_train,
          item_features=item_feat_train,
          epochs=40)

In [ ]:
prediction = model.predict(int)

Посчитаем только auc, так как lightFm не поддерживает log loss

In [12]:
test_model(model, user_item_test, user_feat_test, item_feat_test)

Auc: 0.4673304557800293
